In [1]:
from data import load_us_perm_visas 

X, y = load_us_perm_visas()
print('decision_date' in X.columns)
print('decision_date_year' in X.columns)

False
True


In [2]:
X

,decision_date_year,decision_date_month,employer_name,employer_city,employer_state,job_info_work_city,job_info_work_state,pw_soc_code,pw_unit_of_pay_9089,pw_source_name_9089,pw_soc_title,country_of_citizenship,class_of_admission,pw_level_9089,pw_amount_9089,wage_offer_unit_of_pay_9089
0,2012,2,netsoft usa inc.,new york,ny,new york,ny,15-1031.00,yr,oes,"computer software engineers, applications",missing,j-1,level ii,75629.0,yr
1,2011,12,pinnacle environemntal corp,carlstadt,ny,new york,ny,47-4041.00,yr,oes,hazardous materials removal workers,missing,b-2,level i,37024.0,yr
2,2011,12,"schnabel engineering, inc.",glen allen,va,lutherville,md,17-2051.00,yr,oes,civil engineers,missing,h-1b,level i,47923.0,yr
3,2011,12,ebenezer mission church,flushing,ny,flushing,ny,43-4071.00,hr,oes,file clerks,missing,b-2,level ii,10.97,hr
4,2012,1,albany international corp.,albany,ny,albany,ny,41-9031.00,yr,oes,sales engineers,missing,l-1,level iv,94890.0,yr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374349,2016,12,pomeroy it solutions sales company,hebron,ky,frankfort,ky,15-1132,yr,oes,"software developers, applications",india,h-1b,level iv,"84,635.00",yr
374350,2016,12,siemens plm software inc.,milford,oh,milford,oh,15-1132,yr,oes,"software developers, applications",india,h-1b,level ii,"69,098.00",yr
374351,2016,12,globallogic inc.,san jose,ca,san jose,ca,15-1121,yr,oes,computer systems analysts,india,l-1,level iii,"114,795.00",yr
374352,2016,12,deutsche bank trust company americas,new york,ny,new york,ny,15-1122,yr,other,information security analysts,india,h-1b,missing,"112,354.00",yr


In [3]:
uniques = X['pw_unit_of_pay_9089'].unique()
print(uniques)

['yr' 'hr' 'missing' 'bi' 'mth' 'wk']


In [4]:
from sklearn.model_selection import train_test_split
from tabpfn import TabPFNClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, classification_report, confusion_matrix, recall_score, precision_score

In [19]:
# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

In [5]:
# # Encode categorical features
# cat_cols = X.select_dtypes(include=['object', 'category']).columns
# if len(cat_cols) > 0:
#     encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
#     X[cat_cols] = encoder.fit_transform(X[cat_cols])

X_small = X[:11000]
y_small = y[:11000]

X_train, X_test, y_train, y_test = train_test_split(X_small, y_small, test_size=0.1, random_state=42)

In [ ]:
# Ignore FutureWarning, of which TabPFNClassifier has a lot!
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [8]:
# Initialize a classifier
clf = TabPFNClassifier(ignore_pretraining_limits=True)
clf.fit(X_train, y_train)

TabPFNClassifier(ignore_pretraining_limits=True)

In [9]:
# Predict labels
predictions = clf.predict(X_test)

In [10]:
print("Accuracy", accuracy_score(y_test, predictions))

Accuracy 0.8136363636363636


In [14]:
# Calculate F1 score with macro averaging
print("F1 Score (macro):", f1_score(y_test, predictions, average='macro'))

# Calculate F1 score with macro averaging
print("F1 Score (micro):", f1_score(y_test, predictions, average='micro'))

# Or with weighted averaging
print("F1 Score (weighted):", f1_score(y_test, predictions, average='weighted'))

F1 Score (macro): 0.605492661813614
F1 Score (micro): 0.8136363636363636
F1 Score (weighted): 0.7753420905851193


In [13]:
from sklearn.preprocessing import LabelEncoder

# Convert string labels to numeric values
le = LabelEncoder()
y_test_encoded = le.fit_transform(y_test)
predictions_encoded = le.transform(predictions)

# Now calculate F1 score with numeric labels
print("F1 Score:", f1_score(y_test_encoded, predictions_encoded))

F1 Score: 0.31893687707641194


In [18]:
# # Predict probabilities
# prediction_probabilities = clf.predict_proba(X_test)
# print("ROC AUC:", roc_auc_score(y_test, prediction_probabilities[:, 1]))


print("Accuracy", accuracy_score(y_test, predictions))

# Calculate and print F1 score
print("F1 Score:", f1_score(y_test, predictions, average='macro'))

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, predictions, digits=3, output_dict=True))
print(classification_report(y_test, predictions, digits=3, output_dict=False))

## March 30
# Accuracy 0.83
# ROC AUC: 0.8062494903783433
# Accuracy 0.8136363636363636

## March 31
# Accuracy 0.8136363636363636

Accuracy 0.8136363636363636
F1 Score: 0.605492661813614

Classification Report:
{'certified': {'precision': 0.8279569892473119, 'recall': 0.9668949771689498, 'f1-score': 0.8920484465508162, 'support': 876.0}, 'denied': {'precision': 0.6233766233766234, 'recall': 0.21428571428571427, 'f1-score': 0.31893687707641194, 'support': 224.0}, 'accuracy': 0.8136363636363636, 'macro avg': {'precision': 0.7256668063119676, 'recall': 0.590590345727332, 'f1-score': 0.605492661813614, 'support': 1100.0}, 'weighted avg': {'precision': 0.7862969874700081, 'recall': 0.8136363636363636, 'f1-score': 0.7753420905851193, 'support': 1100.0}}
              precision    recall  f1-score   support

   certified      0.828     0.967     0.892       876
      denied      0.623     0.214     0.319       224

    accuracy                          0.814      1100
   macro avg      0.726     0.591     0.605      1100
weighted avg      0.786     0.814     0.775      1100



In [20]:
def log_classification_report_metrics(report_dict):
    """
    Log each metric from a classification report dictionary to MLflow.
    
    Parameters:
    -----------
    report_dict : dict
        The dictionary returned by sklearn.metrics.classification_report with output_dict=True
    """
    # Loop through the top level dictionary
    for class_key, metrics in report_dict.items():
        # Check if the value is a dictionary (for class metrics)
        if isinstance(metrics, dict):
            # Log metrics for each class
            for metric_name, value in metrics.items():
                # Create a named metric with class and metric info
                metric_full_name = f"{class_key}_{metric_name}"
                mlflow.log_metric(metric_full_name, value)
        else:
            # For overall metrics like accuracy that aren't nested
            mlflow.log_metric(class_key, metrics)

In [ ]:
import mlflow
from mlflow.models import infer_signature

# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

# Create a new MLflow Experiment
mlflow.set_experiment("MLflow Quickstart")

report_dict = classification_report(y_test, predictions, output_dict=True)

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    # mlflow.log_metric("accuracy", accuracy)
    log_classification_report_metrics(report_dict)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "TabPFN basic for US Visa data")

    # # Infer the model signature
    # signature = infer_signature(X_train, clf.predict(X_train))

    # # Log the model
    # model_info = mlflow.sklearn.log_model(
    #     sk_model=clf,
    #     artifact_path="iris_model",
    #     signature=signature,
    #     input_example=X_train,
    #     registered_model_name="tracking-quickstart",
    # )

🏃 View run bustling-bird-629 at: http://127.0.0.1:8080/#/experiments/590767339689717016/runs/46fab84fdcec4d388fa0815199c5f100
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/590767339689717016


## From Dan

### Other models

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=200)
model.fit(X_train, y_train)
predictions_rf = model.predict(X_test)

In [ ]:
# if you don't have GPU and cuda, change task_type
from catboost import CatBoostClassifier
model = CatBoostClassifier(iterations=1000, 
                           task_type="GPU",
                           devices='0:1')#, class_weights=class_weights)
model.fit(X_res,
          y_res)

### evals

In [ ]:
from sklearn.metrics import f1_score, classification_report, confusion_matrix, recall_score, precision_score
predictions = model.predict(X_test)
print(classification_report(y_test, predictions, digits=3))

In [ ]:

from sklearn.metrics import f1_score, classification_report, confusion_matrix

f_scores['RF'] = f1_score(y_test_2, predictions, average='macro')
f_scores_micro['RF'] = f1_score(y_test_2, predictions, average='micro')

accuracy_scores['RF'] = precision_score(y_test_2, predictions, average='macro') #accuracy_score(y_test_2, predictions)
recall_scores['RF'] = recall_score(y_test_2, predictions, average='macro')


# print(f1_score(y_test, predictions, average='macro'))
print(classification_report(y_test_2, predictions, digits=3))
# print(confusion_matrix(y_test, predictions))